In [1]:
import os
import datetime
from glob import iglob,glob

import xarray as xr
import numpy as np
import pandas as pd

from params import *

In [39]:
def raise_incorrect_type(which):
    """
    Raise value error if incorrect forecast type

    Args:
        which (str): kind of forecast (atm, ocean ,wave)
    """
    if which not in ['atm','ocean','wave']:
        raise ValueError('Which parameter must indicate only some\
                          of these ["ocean","wave", "atm"]') 
    return

def first_forecast_path(which):
    """
    Return the first aviable forecast of each type

    Args:
        which (str): kind of forecast (atm, ocean ,wave)

    Returns:
        str: path to netcdf file
    """
    path = glob(eval(f'{which}_globalparams')['forecast_dir']+'/*.nc')
    path = sorted(path)[0]
    return path

def first_forecast_date(which):
    """
    Return the date of the first aviable forecast of each type

    Args:
        which (str): kind of forecast (atm, ocean ,wave)

    Returns:
        str: first forecast date
    """
    path = first_forecast_path(which)
    fdir     = eval(f'{which}_globalparams')['forecast_dir']
    fsuffix  = eval(f'{which}_globalparams')['forecast_suffix']
    date = path.replace(fdir,'')
    date = date.replace(fsuffix,'')
    date = date.replace('/','')
    date = date.replace('.nc','')
    date = pd.to_datetime(date).strftime('%F')
    return date
    

def last_forecast_path(which):
    """
    Return last aviable forecast of each type
    Args:
        which (str): kind of forecast (atm, ocean ,wave)

    Returns:
        str: path to netcdf file
    """
    path = glob(eval(f'{which}_globalparams')['forecast_dir']+'/*.nc')
    path = sorted(path)[-1]

    return path


def forecast_path(date, which):
    """
    Return forecast path for a type and specific date.
    
    Args:
        which (str): type of forecast (atm, ocean ,wave)
        date  (str): %Y-%m-%d

    Returns:
        str: path to netcdf file
    """
    # Some checks
    raise_incorrect_type(which)
    
    date      = pd.to_datetime(date)
    firstdate = pd.to_datetime(first_forecast_date(which)) 
    # If date goes before any aviable forecast raise error
    if date<firstdate:
        raise RuntimeError(f'Forecast are only aviable for dates after {firstdate}')
    # If date is from today or the future return last valid forecast
    if pd.to_datetime(date).date() >= datetime.datetime.now().date():
        path = last_forecast_path(which)
    else:
        fdir     = eval(f'{which}_globalparams')['forecast_dir']
        fsuffix  = eval(f'{which}_globalparams')['forecast_suffix']
        fdformat = eval(f'{which}_globalparams')['forecast_dateformat']
        path     = '{}/{}'.format(fdir,fsuffix)
        path     = path+'{}.nc'.format(date.strftime(fdformat))
    
    # Check if constructed path exist else raise error
    if os.path.isfile(path):
        # All good
        pass
    else:
        raise RuntimeError(f'{path} doesnt exists!')
    
    return path

In [72]:
def load_wrf(path, **kwargs):
    """ 
    Load a wrf model output and adjust coordinates for the package to work.
    Args:
        path (str): path to netcdf file
        **kwargs passed to xarray.open_dataset

    Returns:
        data (xDataset): loaded xarray
    """
    data  = xr.open_dataset(path, **kwargs)
    data  = data.rename({'XLAT':'lat','XLONG':'lon','XTIME':'time'})
    cdict = {value:key for key, value in atm_variables.items()}
    data  = data.rename(cdict)[atm_variables.keys()]
    
    inittime = data.attrs['SIMULATION_START_DATE'].replace('_',' ')
    data.coords['inittime'] = pd.to_datetime(inittime) 
    return data

In [73]:
path = forecast_path('2023-08-09','atm')
wrf  = load_wrf(path)

In [74]:
wrf

<xarray.Dataset>
Dimensions:     (time: 259, y: 300, x: 294)
Coordinates:
    lat         (y, x) float32 ...
    lon         (y, x) float32 ...
  * time        (time) datetime64[ns] 2023-08-08T12:00:00 ... 2023-08-19T06:0...
  * y           (y) float64 -3.65e+06 -3.646e+06 ... -2.458e+06 -2.454e+06
  * x           (x) float64 -5.86e+05 -5.82e+05 -5.78e+05 ... 5.82e+05 5.86e+05
    inittime    datetime64[ns] 2023-08-08T12:00:00
Data variables:
    uwnd        (time, y, x) float32 ...
    vwnd        (time, y, x) float32 ...
    lsm         (time, y, x) float32 ...
    precip      (time, y, x) float32 ...
    rh2         (time, y, x) float32 ...
    t2m         (time, y, x) float32 ...
    lowclouds   (time, y, x) float32 ...
    midclouds   (time, y, x) float32 ...
    highclouds  (time, y, x) float32 ...
Attributes: (12/131)
    TITLE:                            OUTPUT FROM WRF V4.3 MODEL
    START_DATE:                      2023-08-08_12:00:00
    SIMULATION_START_DATE:           2023-08-08_12:00:00
    WEST-EAST_GRID_DIMENSION:        295
    SOUTH-NORTH_GRID_DIMENSION:      301
    BOTTOM-TOP_GRID_DIMENSION:       30
    ...                              ...
    ISLAKE:                          21
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.1